https://huggingface.co/Edresson/wav2vec2-large-100k-voxpopuli-ft-Common-Voice_plus_TTS-Dataset-russian

In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer, Wav2Vec2ForCTC
import torchaudio
import re


In [2]:
tokenizer = AutoTokenizer.from_pretrained("Edresson/wav2vec2-large-100k-voxpopuli-ft-Common-Voice_plus_TTS-Dataset-russian")

model = Wav2Vec2ForCTC.from_pretrained("Edresson/wav2vec2-large-100k-voxpopuli-ft-Common-Voice_plus_TTS-Dataset-russian")

/home/redalexdad/anaconda3/envs/dl_science/lib/python3.9/site-packages/transformers/configuration_utils.py:365: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [3]:
# dataset = load_dataset("common_voice", "ru", split="test", data_dir="./cv-corpus-6.1-2020-12-11", trust_remote_code=True)
dataset = load_dataset("mozilla-foundation/common_voice_11_0", "ru")

resampler = torchaudio.transforms.Resample(orig_freq=48_000, new_freq=16_000)
chars_to_ignore_regex = r"[^A-Za-z0-9(),!?\'\`\"\_\n]"

def map_to_array(batch):
    speech, _ = torchaudio.load(batch["path"])
    batch["speech"] = resampler.forward(speech.squeeze(0)).numpy()
    batch["sampling_rate"] = resampler.new_freq
    batch["sentence"] = re.sub(chars_to_ignore_regex, '', batch["sentence"]).lower().replace("â€™", "'")
    return batch

/home/redalexdad/anaconda3/envs/dl_science/lib/python3.9/site-packages/datasets/load.py:1461: FutureWarning: The repository for mozilla-foundation/common_voice_11_0 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mozilla-foundation/common_voice_11_0
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [10]:
def map_to_pred(batch):
    # Получаем логиты от модели
    logits = model(batch["speech"], attention_mask=batch["attention_mask"]).logits
    
    # Извлекаем индексы максимальных значений для каждого элемента по последней размерности
    predicted_ids = torch.argmax(logits, dim=-1)
    
    # Декодируем индексы в текст, используя процессор
    predicted_text = processor.batch_decode(predicted_ids)
    
    # Избавляемся от лишних символов и приводим к нижнему регистру, если необходимо
    predicted_text = re.sub(chars_to_ignore_regex, '', predicted_text).lower().replace("â€™", "'")
    
    # Возвращаем предсказанный текст
    return {"predicted_text": predicted_text}

In [5]:
ds = dataset.map(map_to_array)

Map:   0%|          | 0/22862 [00:00<?, ? examples/s]

Map:   0%|          | 0/9629 [00:00<?, ? examples/s]

Map:   0%|          | 0/9630 [00:00<?, ? examples/s]

Map:   0%|          | 0/16865 [00:00<?, ? examples/s]

Map:   0%|          | 0/7159 [00:00<?, ? examples/s]

In [8]:
ds['test']

Dataset({
    features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'speech', 'sampling_rate'],
    num_rows: 9630
})

In [9]:
result = ds.map(map_to_pred, batched=True, batch_size=1, remove_columns=list(ds['test'].features.keys()))

Map:   0%|          | 0/22862 [00:00<?, ? examples/s]

NameError: name 'processed_data' is not defined

In [ ]:
print(wer.compute(predictions=result["predicted"], references=result["target"]))